In [ ]:
TRAINER = "diffusers/examples/dreambooth/train_dreambooth.py"
BACK_CONVERTER = "diffusers/scripts/convert_diffusers_to_original_stable_diffusion.py"

In [ ]:
#@title Login to wandb to watch training process (Optional, keep key empty if you want to skip)
WANDB_KEY = ""
if WANDB_KEY != "":
  !wandb login $WANDB_KEY

In [ ]:
#declare your model name
modelName = "animefull-final-pruned"

#default models location
originalModels = "/models"
convertModels = "/convertModels"

# if not need vae, comment it
vae_arg = f"--vae_path {originalModels}/animevae.pt"

#--------default variable
SRC_PATH = originalModels + "/" + modelName
MODEL_NAME = convertModels + "/" + modelName

## Instance Prompt and Class Prompt

What your training set is about|Instance prompt must contain|Class prompt should describe
-|-|-
A object/person|`[V]`|The object's type and/or characteristics
A artist's style|`by [V]`|The common characteristics of the training set

Where:
* `[V]` is a *token* in CLIP's [vocabulary](https://huggingface.co/openai/clip-vit-large-patch14/raw/main/vocab.json) which is not meaningful to the model. `sks` is a great example.

A common pitfall: like if you are training about a specific person with name `[N]`, you should NOT use `[N]` as `[V]`. Names have high chance of being separated (tokenized) to multiple tokens, which is possibly hazardous.

Finally `[V]` will carry the new information learned by the model.

### Examples

Training about a female character:
* Instance prompt: `sks 1girl`
* Class prompt: `1girl`

Training about hatsune miku (don't do this btw, model already knows):
* Instance prompt: `masterpiece, best quality, sks 1girl, aqua eyes, aqua hair`
* Class prompt: `masterpiece, best quality, 1girl, aqua eyes, aqua hair`

Training about an artist's style on drawing female characters:
* Instance prompt: `1girl, by sks`
* Class prompt: `1girl`

In [ ]:
# declare input output folder

trainingFolder = "/train"
trainFolderName = "trcoot"

#--------default variable
INSTANCE_DIR = trainingFolder + "/" + trainFolderName +"/input"
CLASS_DIR = trainingFolder + "/" + trainFolderName +"/class"
OUTPUT_DIR =  trainingFolder + "/" + trainFolderName +"/output"

!mkdir -p $INSTANCE_DIR
!mkdir -p $CLASS_DIR
!mkdir -p $OUTPUT_DIR

print(f"[*] Weights will be saved at {OUTPUT_DIR}")

In [ ]:
INSTANCE_PROMPT = "1girl, by bala, "

CLASS_PROMPT = "1girl"
CLASS_NEGATIVE_PROMPT = "lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry"
NUM_CLASS_IMAGES = 25

SAVE_SAMPLE_PROMPT = "masterpiece, best quality, 1girl, by bala, looking at viewer"
SAVE_SAMPLE_NEGATIVE_PROMPT = "lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry"

In [ ]:
#@title Advanced Parameters
MAX_TRAIN_STEPS = 2000 #@param {type:"number"}
SAVE_INTERVAL = 500 #@param {type:"number"}
SEED = 114514 #@param {type:"number"}
#@markdown ## Data Processing
RESOLUTION = 512 #@param {type:"slider", min:64, max:2048, step:28}
ASPECT_RATIO_BUCKETING = False #@param {type:"boolean"}
READ_PROMPT_FROM_TXT = "instance" #@param ["no", "instance", "class", "both"] {allow-input: false}
#@markdown ## Forward Pass
TRAIN_BATCH_SIZE = 1 #@param {type:"slider", min:1, max:10, step:1}
GRADIENT_ACCUMULATION_STEPS = 1 #@param {type:"slider", min:1, max:10, step:1}
CLIP_SKIP = 2 #@param {type:"slider", min:1, max:6, step:1}
MIXED_PRECISION = "fp16" #@param ["no", "fp16", "bf16"] {allow-input: false}
#@markdown ## Optimizer / Backward Pass
OPTIMIZER = "adamw_8bit" #@param ["adamw", "adamw_8bit", "adamw_ds", "sgdm", "sgdm_8bit"] {allow-input: false}
LEARNING_RATE = 5e-6 #@param {type:"number"}
LR_SCHEDULER = "cosine_with_restarts"  #@param ["linear", "cosine", "cosine_with_restarts", "polynomial", "constant", "constant_with_warmup"] {allow-input: false}
LR_WARMUP_STEPS = 100 #@param {type:"number"}
LR_CYCLES = 1 #@param {type:"number"}
LAST_EPOCH = -1 #@param {type:"number"}
SCALE_LR = True #@param {type:"boolean"}
PRIOR_PRESERVATION = False #@param {type:"boolean"}
PRIOR_LOSS_WEIGHT = 1 #@param {type:"slider", min:0, max:1, step:0.01}
#@markdown ## Inference (Class Set Generation / Sample Images Generation)
INFER_STEPS = 28 #@param {type:"integer"}
GUIDANCE_SCALE = 11 #@param {type:"integer"}
SAMPLE_N = 4  #@param {type:"integer"}
INFER_BATCH_SIZE = 1 #@param {type:"slider", min:1, max:10, step:1}

In [ ]:
%%bash

mkdir -p ~/.cache/huggingface/accelerate

cat > ~/.cache/huggingface/accelerate/default_config.yaml <<- EOM
compute_environment: LOCAL_MACHINE
deepspeed_config: {}
distributed_type: 'NO'
downcast_bf16: 'no'
fsdp_config: {}
machine_rank: 0
main_process_ip: null
main_process_port: null
main_training_function: main
mixed_precision: fp16
num_machines: 1
num_processes: 1
use_cpu: false
EOM

In [ ]:
%cd /content
!mkdir -p $OUTPUT_DIR

wandb_arg = "--wandb" if WANDB_KEY != "" else ""
scale_lr_arg = "--scale_lr" if SCALE_LR else ""
ppl_arg = f"--with_prior_preservation --prior_loss_weight={PRIOR_LOSS_WEIGHT}" if PRIOR_PRESERVATION else ""
read_prompt_arg = f"--read_prompt_from_txt {READ_PROMPT_FROM_TXT}" if READ_PROMPT_FROM_TXT != "no" else ""
arb_arg = "--use_aspect_ratio_bucket --debug_arb" if ASPECT_RATIO_BUCKETING else ""


!accelerate launch $TRAINER \
  --instance_data_dir "{INSTANCE_DIR}" \
  --instance_prompt "{INSTANCE_PROMPT}" \
  --pretrained_model_name_or_path "{MODEL_NAME}" \
  --pretrained_vae_name_or_path "{MODEL_NAME}/vae" \
  --output_dir "{OUTPUT_DIR}" \
  --seed=$SEED \
  --resolution=$RESOLUTION \
  --optimizer "{OPTIMIZER}" \
  --train_batch_size=$TRAIN_BATCH_SIZE \
  --learning_rate=$LEARNING_RATE \
  --lr_scheduler=$LR_SCHEDULER \
  --lr_warmup_steps=$LR_WARMUP_STEPS \
  --lr_cycles=$LR_CYCLES \
  --last_epoch=$LAST_EPOCH \
  --max_train_steps=$MAX_TRAIN_STEPS \
  --save_interval=$SAVE_INTERVAL \
  --class_data_dir "{CLASS_DIR}" \
  --class_prompt "{CLASS_PROMPT}" --class_negative_prompt "{CLASS_NEGATIVE_PROMPT}" \
  --num_class_images=$NUM_CLASS_IMAGES \
  --save_sample_prompt "{SAVE_SAMPLE_PROMPT}" --save_sample_negative_prompt "{SAVE_SAMPLE_NEGATIVE_PROMPT}" \
  --n_save_sample=$SAMPLE_N \
  --infer_batch_size=$INFER_BATCH_SIZE \
  --infer_steps=$INFER_STEPS \
  --guidance_scale=$GUIDANCE_SCALE \
  --gradient_accumulation_steps=$GRADIENT_ACCUMULATION_STEPS \
  --gradient_checkpointing \
  --save_unet_half \
  --mixed_precision "{MIXED_PRECISION}" \
  --clip_skip=$CLIP_SKIP \
  --sample_interval=$SAVE_INTERVAL \
  $wandb_arg $scale_lr_arg $ppl_arg $read_prompt_arg $arb_arg

# disabled: --not_cache_latents 

## Convert weights to ckpt to use in web UIs like AUTOMATIC1111.

In [ ]:
#@markdown Which step number to use.
use_checkpoint = '2000' #@param {type:"string"}
#@markdown Id of which run to use (empty = latest run).
run_id = '' #@param {type:"string"}

if not run_id:
  runs = [d for d in Path(OUTPUT_DIR).iterdir() if d.is_dir()]
  runs.sort(lambda d: d.stat().st_ctime, reverse=True)
  run_id = runs[0].name

ckpt_path = f'{OUTPUT_DIR}/{run_id}/{use_checkpoint}/model.ckpt'

# You can add --vae and --text_encoder if you want.
!python "{BACK_CONVERTER}" --model_path "{OUTPUT_DIR}/{run_id}/{use_checkpoint}" --checkpoint_path $ckpt_path \
  --unet_dtype fp16

print(f"[*] Converted ckpt saved at {ckpt_path}")

## Misc

In [ ]:
!git clone https://github.com/CCRcmcpe/diffusers.git